In [1]:
import zlib
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
def entropy_decoder(compressed_img,original_shape):
    decompressed = zlib.decompress(compressed_img)
    ẑ = np.frombuffer(decompressed, dtype=np.int8)
    ẑ = ẑ.reshape(original_shape)
    return ẑ
    
def batch_entropy_decode(compressed_batch,original_size):
    z = torch.cat([
        torch.tensor(
            entropy_decoder(compressed_batch[i], original_size)
        ) 
        for i in range(batch_size)
    ])
    return z

def live_plot(x):
    plt.clf()
    plt.plot(x)
    disp.display(plt.gcf())
    disp.clear_output(wait=True)

In [12]:
class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        
        self.conv1 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(512, 1024, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(1024, 2048, kernel_size=3, stride=2, padding=1)
        self.conv5 = nn.Conv2d(2048, 4096, kernel_size=3, stride=2, padding=1)
        self.conv6 = nn.Conv2d(4096, 8192, kernel_size=3, stride=2, padding=1)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))  # adaptive pooling to 1x1
        self.conv7 = nn.Conv2d(8192, 16384, kernel_size=1)  # kernel size 1 to maintain 1x1 size
        self.conv8 = nn.Conv2d(16384, 1001, kernel_size=1)  # kernel size 1 to maintain 1x1 size

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool(x)  # apply adaptive pooling
        x = F.relu(self.conv7(x))
        x = self.conv8(x)  # no activation on the last layer
        return x

In [13]:
dataset = load_dataset("danjacobellis/imagenet_RDAE_batched_250k",split='train').with_format("torch")
N_classes=1001;
net = FCN().to("cuda")
parameters = set(p for n, p in net.named_parameters() if not n.endswith(".quantiles"))
optimizer = optim.Adam(parameters, lr=1e-4)

In [17]:
for i in range(5):
    for (i_batch, batch) in enumerate(dataset):
        latent_size = batch['latent_size']
        batch_size = latent_size[0];
        original_size = latent_size.numpy().copy();
        original_size[0] = 1;
        
        compressed_batch = batch['compressed_batch']
        z = batch_entropy_decode(compressed_batch,original_size);
        z = z.to("cuda").to(torch.float)
        
        label = batch['label'].to(torch.int64)
        y = torch.zeros((batch_size,N_classes,1,1),dtype=torch.int16)
        y.scatter_(1, label.view(-1, 1, 1, 1), 1)
        y = y.to("cuda").to(torch.float)
    
        out = net(z)
        out = out.squeeze(-1).squeeze(-1)
        y = y.squeeze(-1).squeeze(-1)
        loss_fn = nn.BCEWithLogitsLoss()
        loss = loss_fn(out, y.float()) 
        
        loss.backward()
        optimizer.step()
    
        torch.save({
                'epoch': i,
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, f"classifier_checkpoint.pth")
        live_plot(loss)
        
        z.detach()
        y.detach()

NameError: name 'rate' is not defined